### 4.6.5 全体のコード

In [0]:
import numpy as np
# import cupy as np  # GPUの場合
import matplotlib.pyplot as plt

# -- 各設定値 --
n_time = 8  # 時系列の数（2進数の桁数）
n_in = 2  # 入力層のニューロン数
n_mid = 32  # 中間層のニューロン数
n_out = 1  # 出力層のニューロン数

eta = 0.01  # 学習係数
n_learn = 5001  # 学習回数
interval = 500  # 経過の表示間隔

# -- 2進数を作成 --
max_num = 2**n_time  # 10進数の上限
binaries = np.zeros((max_num, n_time), dtype=int)  # 2進数を格納する配列
for i in range(max_num):
    num10 = i  # 10進数の数
    for j in range(n_time):
        pow2 = 2 ** (n_time-1-j)  # 2の累乗
        binaries[i, j] = num10 // pow2
        num10 %= pow2
# print(binaries)

# -- RNN層 -- 
class SimpleRNNLayer:
    def __init__(self, n_upper, n):
        # パラメータの初期値
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)  # Xavierの初期値
        self.v = np.random.randn(n, n) / np.sqrt(n)  # Xavierの初期値
        self.b = np.zeros(n)

    def forward(self, x, y_prev):  # y_prev: 前の時刻の出力
        u = np.dot(x, self.w) + np.dot(y_prev, self.v) + self.b
        self.y = np.tanh(u)  # 出力
    
    def backward(self, x, y, y_prev, grad_y):
        delta = grad_y * (1 - y**2)

        # 各勾配
        self.grad_w += np.dot(x.T, delta)
        self.grad_v += np.dot(y_prev.T, delta)
        self.grad_b += np.sum(delta, axis=0)

        self.grad_x = np.dot(delta, self.w.T)
        self.grad_y_prev = np.dot(delta, self.v.T)

    def reset_sum_grad(self):
        self.grad_w = np.zeros_like(self.w)
        self.grad_v = np.zeros_like(self.v)
        self.grad_b = np.zeros_like(self.b)

    def update(self, eta):
        self.w -= eta * self.grad_w
        self.v -= eta * self.grad_v
        self.b -= eta * self.grad_b

# -- 全結合 出力層 --
class RNNOutputLayer:
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)  # Xavierの初期値
        self.b = np.zeros(n)

    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = 1/(1+np.exp(-u))  # シグモイド関数

    def backward(self, x, y, t):
        delta = (y-t) * y * (1-y)
        
        self.grad_w += np.dot(x.T, delta)
        self.grad_b += np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T) 

    def reset_sum_grad(self):
        self.grad_w = np.zeros_like(self.w)
        self.grad_b = np.zeros_like(self.b)

    def update(self, eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

# -- 各層の初期化 --
rnn_layer = SimpleRNNLayer(n_in, n_mid)
output_layer = RNNOutputLayer(n_mid, n_out)

# -- 訓練 --
def train(x_mb, t_mb):
    # 各出力を格納する配列
    y_rnn = np.zeros((len(x_mb), n_time+1, n_mid))
    y_out = np.zeros((len(x_mb), n_time, n_out))

    # 順伝播
    y_prev = y_rnn[:, 0, :]
    for i in range(n_time):
        # RNN層
        x = x_mb[:, i, :]
        rnn_layer.forward(x, y_prev)
        y = rnn_layer.y
        y_rnn[:, i+1, :] = y
        y_prev = y

        # 出力層
        output_layer.forward(y)
        y_out[:, i, :] = output_layer.y

    # 逆伝播
    output_layer.reset_sum_grad()
    rnn_layer.reset_sum_grad()
    grad_y = 0
    for i in reversed(range(n_time)):
        # 出力層
        x = y_rnn[:, i+1, :]
        y = y_out[:, i, :]
        t = t_mb[:, i, :]
        output_layer.backward(x, y, t)
        grad_x_out = output_layer.grad_x

        # RNN層
        x = x_mb[:, i, :]
        y = y_rnn[:, i+1, :]
        y_prev = y_rnn[:, i, :]
        rnn_layer.backward(x, y, y_prev, grad_y+grad_x_out)
        grad_y = rnn_layer.grad_y_prev

    # パラメータの更新
    rnn_layer.update(eta)
    output_layer.update(eta)

    return y_out

# -- 誤差を計算 --
def get_error(y, t):
    return 1.0/2.0*np.sum(np.square(y - t))  # 二乗和誤差

for i in range(n_learn):
    # -- ランダムな10進数 --
    num1 = np.random.randint(max_num//2)
    num2 = np.random.randint(max_num//2)

    # -- 入力を用意 --
    x1= binaries[num1]
    x2= binaries[num2]
    x_in = np.zeros((1, n_time, n_in))
    x_in[0, :, 0] = x1
    x_in[0, :, 1] = x2
    x_in  = np.flip(x_in, axis=1)  # 桁が小さい方を古い時刻に

    # -- 正解を用意 --
    t = binaries[num1+num2]
    t_in = t.reshape(1, n_time, n_out)
    t_in = np.flip(t_in , axis=1)

    # -- 訓練 --
    y_out = train(x_in, t_in)
    y = np.flip(y_out, axis=1).reshape(-1)

    # -- 誤差を求める --
    error = get_error(y_out, t_in)

    # -- 経過の表示 -- 
    if i%interval == 0:
        y2 = np.where(y<0.5, 0, 1)  # 2進数の結果
        y10 = 0  # 10進数の結果
        for j in range(len(y)):
            pow2 = 2 ** (n_time-1-j)  # 2の累乗
            y10 += y2[j] * pow2

        print("n_learn:", i)
        print("error:", error)
        print("output :", y2)
        print("correct:", t)

        c = "\(^_^)/ : " if (y2 == t).all() else "orz : "
        print(c + str(num1) + " + " + str(num2) + " = " + str(y10))
        print("-- -- -- -- -- -- -- -- -- -- -- -- -- -- --")